In [1]:
import numpy as np

## Входные данные:

1. radius, length = длина канала
2. magma = начальная температура магмы (характеристика из монщности нагрева)
3. final_temparure = неиронично это финальная температура
4. alpha = коэф-т теплопроводности

In [3]:
radius, length = map(int, input().split())
magma = int(input())
final_temperature = 70
alpha = 0.56

print(radius, length, magma)

100 100 700


Будем двигаться по радиусу с шагом 10, а по времени замерим 10000 условных единиц времени

In [86]:
count_r = radius // 10
count_t = 1000

Сетка, по которой будем осуществлять вычисления. L: сетка разбиения нашего цилиндра. Разбивать будем на 10 частей

In [150]:
L = np.ndarray((10, 1000, 10))

In [112]:
print(L)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


Предположим, что на каждом кольце изначальное состояние температуры случайное. Зададим в районе 100 градусов, чтобы она не испарялась. Крайнее значение уменьшается на 10% (опционально) потому что отдает тепло в землю

In [160]:
c = 100
for i in range(len(L)):
    L[i][0] = np.random.normal(c, 10, count_r)
    L[i][0, 0] = L[i][0, 0] * 0.9 
    c -= 3


In [94]:
L[0]

array([[ 91.41724205,  93.76180923, 102.91032305, ...,  95.69564922,
        101.95175304, 100.02837706],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

Теперь наше уравнение. По сетке оно выглядит вот так

$T_{i,j+1} = T_{ij} + \frac{\alpha \Delta t}{r_i \Delta r^2} (T_{i+1,j} - 2T_{ij} + T_{i-1, j}) $

где i = индекс по радиусу, j = индекс по времени.

То есть мы имеем i * j состояний. 

Так мы можем посчитать для всех внутренних точек, и для каждой внутренней точки перейти в следующее состояние T[i, 1]. Но на границах будет проблема: для них неизвестны i+1 или i-1. Я буду рассчитывать немного криво: например, зная для точки только i+1 значение, ему же буду приравнивать i-1. 

TODO: доделать (придумать что делать с крайними точками)

In [161]:
# Шаг по времени
dt = 100

In [101]:
count_r-1

9

In [162]:
for el in range(len(L)): # Каждое кольцо 
    for j in range(1, count_t-1): # каждый промежуток времени
        L[el][j,0] = (L[el][j-1, 0] + ((alpha * dt)/(count_r * (count_r**2))) * (L[el][j-1, 1] - 2*(L[el][j-1, 0]) + L[el][j-1, 1])) * 0.83
        for i in range(1, count_r - 1): # каждый радиус
            L[el][j, i] = L[el][j-1, i] + ((alpha * dt)/((i*count_r) * (count_r**2))) * (L[el][j-1, i+1] - 2*(L[el][j-1, i]) + L[el][j-1, i-1])
        L[el][j, -1] = L[el][j-1, -1] + ((alpha * dt)/((9*count_r) * (count_r**2))) * (L[el][j-1, 8] - 2*(L[el][j-1, 9]) + L[el][j-1, 8])


Не остывает, поскольку как я понимаю, берет температуру у соседнего (так называемый *теплообмен*)Посмотрим что со всеми остальными

In [166]:
for i in range(len(L)):
    print(L[i][10][4])

111.03165347349314
100.74692397213852
102.3333062078945
86.21822123875401
88.11124742583614
90.72790001362328
77.7650792595319
74.61584200230152
70.761621442414
63.0099603274425


Получается, что где-то нагревается, а где-то остывает. Тут есть небольшой коэффициент, поясню:

0.97 = температура уменьшается на краю канала, поскольку она отдает тепло в землю
